<a href="https://colab.research.google.com/github/sotirischatzimiltis/PhD/blob/main/Code/ITU_Competition_Intrusion_and_Vulnerability_Detection_in_Software_Defined_Networks(SDN)/ITU_competition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# mount drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [27]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder

# Read Files and Basic Pre-processing

In [22]:
train_file_path = '/content/drive/MyDrive/ITU_Competition_Intrusion_and_Vulnerability_Detection_in_Software_Defined_Networks(SDN)/Train_ULAK.csv'
test_file_path = '/content/drive/MyDrive/ITU_Competition_Intrusion_and_Vulnerability_Detection_in_Software_Defined_Networks(SDN)/Test_ULAK.csv'

train_df = pd.read_csv(train_file_path,skipinitialspace=True)
test_df = pd.read_csv(test_file_path,skipinitialspace=True)

if train_df is not None and test_df is not None:
    print("Data loaded successfully!")
    print("Train Data:")
    print(train_df.head())  # Display the first few rows of the train DataFrame
    print("\nTest Data:")
    print(test_df.head())   # Display the first few rows of the test DataFrame

Data loaded successfully!
Train Data:
   Destination Port  Flow Duration  Total Fwd Packets  Total Backward Packets  \
0                80       68855579                 10                       6   
1                53            196                  2                       2   
2               123            118                  2                       2   
3                80         295657                  7                      10   
4                80       15705351                  7                       5   

   Total Length of Fwd Packets  Total Length of Bwd Packets  \
0                         1038                        11595   
1                           70                          174   
2                           96                           96   
3                         1114                        15841   
4                          407                          452   

   Fwd Packet Length Max  Fwd Packet Length Min  Fwd Packet Length Mean  \
0                    

In [17]:
#train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1783356 entries, 0 to 1783355
Data columns (total 79 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0    Destination Port             int64  
 1    Flow Duration                int64  
 2    Total Fwd Packets            int64  
 3    Total Backward Packets       int64  
 4   Total Length of Fwd Packets   int64  
 5    Total Length of Bwd Packets  int64  
 6    Fwd Packet Length Max        int64  
 7    Fwd Packet Length Min        int64  
 8    Fwd Packet Length Mean       float64
 9    Fwd Packet Length Std        float64
 10  Bwd Packet Length Max         int64  
 11   Bwd Packet Length Min        int64  
 12   Bwd Packet Length Mean       float64
 13   Bwd Packet Length Std        float64
 14  Flow Bytes/s                  float64
 15   Flow Packets/s               float64
 16   Flow IAT Mean                float64
 17   Flow IAT Std                 float64
 18   Flow IAT Max         

In [35]:
pd.set_option('display.max_rows', None)
train_df.isnull().sum()
# drop 859 records with null values in the Flow Bytes field
train_df = train_df.dropna()


In [19]:
#test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 512077 entries, 0 to 512076
Data columns (total 79 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0    Destination Port             512077 non-null  int64  
 1    Flow Duration                512077 non-null  int64  
 2    Total Fwd Packets            512077 non-null  int64  
 3    Total Backward Packets       512077 non-null  int64  
 4   Total Length of Fwd Packets   512077 non-null  int64  
 5    Total Length of Bwd Packets  512077 non-null  int64  
 6    Fwd Packet Length Max        512077 non-null  int64  
 7    Fwd Packet Length Min        512077 non-null  int64  
 8    Fwd Packet Length Mean       512077 non-null  float64
 9    Fwd Packet Length Std        512077 non-null  float64
 10  Bwd Packet Length Max         512077 non-null  int64  
 11   Bwd Packet Length Min        512077 non-null  int64  
 12   Bwd Packet Length Mean       512077 non-nul

In [20]:
pd.set_option('display.max_rows', None)
test_df.isnull().sum()
# drop 249 records with null values in Flow Bytes field
test_df = test_df.dropna()

In [21]:
# Summary statistics of the dataset
train_df.describe()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
count,1.782497e+06,1.782497e+06,1.782497e+06,1.782497e+06,1.782497e+06,1.782497e+06,1.782497e+06,1.782497e+06,1.782497e+06,1.782497e+06,...,1.782497e+06,1.782497e+06,1.782497e+06,1.782497e+06,1.782497e+06,1.782497e+06,1.782497e+06,1.782497e+06,1.782497e+06,1.782497e+06
mean,8.064779e+03,1.480013e+07,9.296903e+00,1.028180e+01,5.433412e+02,1.606857e+04,2.078776e+02,1.871320e+01,5.825356e+01,6.897491e+01,...,5.284454e+00,-2.232803e+03,8.153444e+04,4.110222e+04,1.530639e+05,5.826710e+04,8.327648e+06,5.072467e+05,8.709600e+06,7.929161e+06
std,1.827769e+04,3.366475e+07,7.233799e+02,9.561413e+02,5.844164e+03,2.218771e+06,7.175694e+02,6.029033e+01,1.861669e+02,2.807168e+02,...,6.019447e+02,9.336271e+05,6.486641e+05,3.884671e+05,1.021356e+06,5.789220e+05,2.364587e+07,4.624571e+06,2.438775e+07,2.337753e+07
min,0.000000e+00,-1.300000e+01,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,-5.368707e+08,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,5.300000e+01,1.550000e+02,2.000000e+00,1.000000e+00,1.200000e+01,2.000000e+00,6.000000e+00,0.000000e+00,6.000000e+00,0.000000e+00,...,0.000000e+00,2.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,8.000000e+01,3.133400e+04,2.000000e+00,2.000000e+00,6.200000e+01,1.230000e+02,3.700000e+01,2.000000e+00,3.400000e+01,0.000000e+00,...,1.000000e+00,2.400000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,4.430000e+02,3.249799e+06,5.000000e+00,4.000000e+00,1.870000e+02,4.840000e+02,8.100000e+01,3.600000e+01,5.000000e+01,2.616295e+01,...,2.000000e+00,3.200000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,6.553400e+04,1.200000e+08,2.177970e+05,2.895850e+05,1.827335e+06,6.396506e+08,2.482000e+04,2.325000e+03,5.675444e+03,7.125597e+03,...,2.074090e+05,1.380000e+02,1.100000e+08,7.420000e+07,1.100000e+08,1.100000e+08,1.200000e+08,7.690000e+07,1.200000e+08,1.200000e+08


In [36]:
# number of each label type train df
train_df['Label'].value_counts()

BENIGN                        1431790
DoS Hulk                       144976
PortScan                       100125
DDoS                            80656
DoS GoldenEye                    6484
FTP-Patator                      5000
SSH-Patator                      3714
DoS slowloris                    3651
DoS Slowhttptest                 3464
Bot                              1238
Web Attack � Brute Force          949
Web Attack � XSS                  410
Infiltration                       22
Web Attack � Sql Injection         12
Heartbleed                          6
Name: Label, dtype: int64

In [25]:
# number of each label type test df
test_df['Label'].value_counts()

BENIGN                        411203
DoS Hulk                       41801
PortScan                       28751
DDoS                           23160
DoS GoldenEye                   1861
FTP-Patator                     1436
SSH-Patator                     1067
DoS slowloris                   1048
DoS Slowhttptest                 994
Bot                              355
Web Attack � Brute Force         272
Web Attack � XSS                 117
Infiltration                       6
Web Attack � Sql Injection         4
Heartbleed                         2
Name: Label, dtype: int64

# Pre-processing of Data

One-Hot-Encoding to Replace Label Values with Columns

In [41]:
labels = train_df['Label'].values.reshape(-1, 1)
print(labels)
# Initialize the OneHotEncoder
encoder = OneHotEncoder()
one_hot_labels = encoder.fit_transform(labels).toarray()
feature_names = encoder.get_feature_names_out(input_features=['Label'])
one_hot_df = pd.DataFrame(one_hot_labels, columns=feature_names)
df_encoded = pd.concat([train_df, one_hot_df], axis=1)
df_encoded = df_encoded.drop('Label', axis=1)

[['DoS Hulk']
 ['BENIGN']
 ['BENIGN']
 ...
 ['BENIGN']
 ['BENIGN']
 ['BENIGN']]


Correlation Analysis

In [42]:
print(df_encoded.head())
correlation_matrix = df_encoded.corr()# Calculate the correlation matrix
print(correlation_matrix)

   Destination Port  Flow Duration  Total Fwd Packets  Total Backward Packets  \
0              80.0     68855579.0               10.0                     6.0   
1              53.0          196.0                2.0                     2.0   
2             123.0          118.0                2.0                     2.0   
3              80.0       295657.0                7.0                    10.0   
4              80.0     15705351.0                7.0                     5.0   

   Total Length of Fwd Packets  Total Length of Bwd Packets  \
0                       1038.0                      11595.0   
1                         70.0                        174.0   
2                         96.0                         96.0   
3                       1114.0                      15841.0   
4                        407.0                        452.0   

   Fwd Packet Length Max  Fwd Packet Length Min  Fwd Packet Length Mean  \
0                  346.0                    0.0            

Print correlation for each type of attack

In [48]:
# Get the correlation coefficients with the target label
for label in feature_names:
    print(label)
    correlation_with_target = correlation_matrix[label]
    # Sort the features based on their correlation with the target label
    sorted_correlation = correlation_with_target.abs().sort_values(ascending=False)
    # Print the correlation coefficients for each feature with the target label
    print("Correlation with Label:")
    print(sorted_correlation)

Label_BENIGN
Correlation with Label:
Label_BENIGN                        1.000000
Label_DoS Hulk                      0.601204
Label_PortScan                      0.492921
Label_DDoS                          0.439872
Label_DoS GoldenEye                 0.122086
Label_FTP-Patator                   0.107164
Label_SSH-Patator                   0.092327
Label_DoS slowloris                 0.091539
Label_DoS Slowhttptest              0.089159
Label_Bot                           0.053268
Label_Web Attack � Brute Force      0.046634
Label_Web Attack � XSS              0.030647
Label_Infiltration                  0.007099
Label_Web Attack � Sql Injection    0.005243
Label_Heartbleed                    0.003707
Fwd URG Flags                       0.001984
CWE Flag Count                      0.001984
FIN Flag Count                      0.001443
Fwd Packets/s                       0.001271
Active Max                          0.001250
Flow Packets/s                      0.001201
Active Std        

Pairplot to visualize relationship between features

In [ ]:
# Pairplot (scatter plot matrix) to visualize relationships between features
sns.pairplot(train_data_frame)
plt.show()

Random Forest for Feature Importance

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import accuracy_score, classification_report, multilabel_confusion_matrix
# Last column is the label column
X_train = train_data_frame.drop(train_data_frame.columns[-1], axis=1)
y_train = train_data_frame[train_data_frame.columns[-1]]

X_test = test_data_frame.drop(test_data_frame.columns[-1], axis=1)
y_test = test_data_frame[test_data_frame.columns[-1]]


# Initialize the Decision Tree classifier
clf = DecisionTreeClassifier()

# Train the classifier on the training data
clf.fit(X_train, y_train)

# Make predictions on the test data
y_pred = clf.predict(X_test)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Calculate precision, recall, and F1-score
report = classification_report(y_test, y_pred)
print("Classification Report:")
print(report)

# Calculate the confusion matrix
conf_matrix = multilabel_confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

# Plot the confusion matrix as a heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", square=True,
            xticklabels=clf.classes_, yticklabels=clf.classes_)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()

# Training of Machine Learning Models